In [1]:
#!/usr/bin/python
from configparser import ConfigParser
import os
import psycopg2
import logging
# from BaseError import NotFoundError, ValidationError, handle_error, default_error_handler,ServerError,ConnectionError

/home/algoscale/Documents/Izenda/izenda_new_env/new_env/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [5]:
# # command = "SELECT * FROM config_table"

# def setPsqlParams():
#     pgsqlParser = ConfigParser()
#     configFile = (os.path.join(os.getcwd(),'config','pgsql_config.ini'))
#     pgsqlParser.read(configFile)
#     hostname = pgsqlParser.get('psql', 'host')
#     username = pgsqlParser.get('psql', 'user')
#     password = pgsqlParser.get('psql', 'passwd')
#     database = pgsqlParser.get('psql', 'database')
#     table = pgsqlParser.get('psql', 'table')
#     return hostname,username,password,database,table


# def getPsqlParams():
#     # print(hostname)
#     hostname,username,password,database,table = setPsqlParams()
#     hostnameString = bytes(hostname, "utf-8").decode("unicode_escape")
#     return hostnameString,username,password,database

# def getTableName():
#     hostname,username,password,database,table = setPsqlParams()
#     return table

# def doQuery(command):
#     hostname,username,password,database,table = setPsqlParams()
#     myConnection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database )
#     cur = myConnection.cursor()
#     cur.execute(command)
#     try:
#         result = cur.fetchall()
#         myConnection.close()
#         return result
#     except psycopg2.ProgrammingError as e:
#         myConnection.commit()
#         myConnection.close()
#         pass

# def getRowCount(value):
#     select_query = "SELECT model_name FROM config_table WHERE model_name = '{}';".format(value) 
#     result = doQuery(select_query)
#     return len(result)

# def insertQuery(values,model_type):
#     if(model_type == 'predict'):
#         insert_query = "INSERT INTO config_table (model_name, model_type,list_of_features,target_column,RMSE,Percent_Error) VALUES ('{}','{}','{}','{}',{},{})".format(*values)
#     elif(model_type == 'classify'):
#         insert_query = "INSERT INTO config_table (model_name, model_type,list_of_features,target_column,accuracy) VALUES ('{}','{}','{}','{}',{})".format(*values)
#     doQuery(insert_query)

# def deleteQuery(value) :
#     delete_query = "DELETE FROM config_table where model_name = '{}'".format(value)
#     doQuery(delete_query)

In [1]:
#!/usr/bin/python
from configparser import ConfigParser
import os
import psycopg2
import logging
# from scripts.BaseError import NotFoundError, ValidationError, handle_error, default_error_handler,ServerError,ConnectionError

connection = None

def connect_database(hostname, username, password, database):
    global connection
    if connection == None:
        try:
            connection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database )
        except:
            connection = None   
    return connection

def init_database(hostname, username, password, database):
    global connection
    conn = connect_database(hostname, username, password, database)
    if conn == None:
        print ("error connecting..")
    else:
        cur = conn.cursor()
        try:
            table = getTableName()
            cur.execute("CREATE TABLE IF NOT EXISTS {}(id serial PRIMARY KEY,model_name VARCHAR UNIQUE NOT NULL,model_type VARCHAR NOT NULL,list_of_features TEXT NOT NULL,target_column VARCHAR NOT NULL,rmse FLOAT,percent_error FLOAT,accuracy FLOAT,prec FLOAT,source VARCHAR NOT NULL);".format(table))
            print("execution complete")
        except:
            print("error creating the table")
        conn.commit()
        conn.close()
        cur.close()
    connection = None 

def setPsqlParams():
    pgsqlParser = ConfigParser()
    configFile = (os.path.join(os.getcwd(),'config','pgsql_config.ini'))
    pgsqlParser.read(configFile)
    hostname = pgsqlParser.get('psql', 'host')
    username = pgsqlParser.get('psql', 'user')
    password = pgsqlParser.get('psql', 'passwd')
    database = pgsqlParser.get('psql', 'database')
    table = pgsqlParser.get('psql', 'table')
    return hostname,username,password,database,table


def getPsqlParams():
    pgsqlParser = ConfigParser()
    configFile = (os.path.join(os.getcwd(),'config','pgsql_config.ini'))
    pgsqlParser.read(configFile)
    hostname = pgsqlParser.get('psql', 'host')
    username = pgsqlParser.get('psql', 'user')
    password = pgsqlParser.get('psql', 'passwd')
    database = pgsqlParser.get('psql', 'database')
    table = pgsqlParser.get('psql', 'table')
    # hostname, username, password, database, table = setPsqlParams()
    hostnameString = bytes(hostname, "utf-8").decode("unicode_escape")
    return hostnameString,username,password,database

def getTableName():
    hostname,username,password,database,table = setPsqlParams()
    return table

def doQuery(command) :
    global connection
    hostname,username,password,database,table = setPsqlParams()
    myconnection = connect_database(hostname, username, password, database)
    cur = myconnection.cursor()
    cur.execute(command)
    if("SELECT" in command):
        result = cur.fetchall()
        myconnection.close()
        cur.close()
        connection = None
        return result
    elif(("INSERT" in command) or ("DELETE" in command)):
        # cur.commit()
        myconnection.commit()
        myconnection.close()
        cur.close()
        connection = None

def getRowCount(value):
    table = getTableName()
    select_query = "SELECT model_name FROM {} WHERE model_name = '{}';".format(table,value) 
    result = doQuery(select_query)
    return len(result)

def insertQuery(values):
    model_type = values[1]
    table = getTableName()
    if(model_type == 'predict'):
        insert_query = "INSERT INTO {} (model_name, model_type,list_of_features,target_column,RMSE,Percent_Error) VALUES ('{}','{}','{}','{}',{},{})".format(table,*values)
    elif(model_type == 'classify'):
        insert_query = "INSERT INTO {} (model_name, model_type,list_of_features,target_column,accuracy) VALUES ('{}','{}','{}','{}',{})".format(table,*values)
    doQuery(insert_query)

def deleteQuery(value) :
    table = getTableName()
    delete_query = "DELETE FROM {} where model_name = '{}'".format(table,value)
    doQuery(delete_query)

/home/algoscale/Documents/Izenda/izenda_new_env/new_env/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# Select Query

In [13]:
res = doQuery('SELECT * from config_table')

In [14]:
res

[(7, 'c', None, None, None, None, None, None, None, None),
 (9, 'fhghf', None, None, None, None, None, None, None, None),
 (11, 'fhf', 'sgs', None, None, None, None, None, None, None),
 (13, 'ff', 'sgs', None, None, None, None, None, None, None),
 (16, 'fdjshsdbjbjhb', None, None, None, None, None, None, None, None),
 (18, 'ashsh', 'b', 'c', "['d','d1']", None, None, 0.9, None, None),
 (19, 'a', 'b', 'c', 'd', None, None, 8.0, None, None),
 (26, 'ard', 'b', 'c', 'd', None, None, 8.0, None, None),
 (30, 'n', 'b', 'c', 'd', 4.0, 8.0, None, None, None),
 (46, 'b167', 'classify', 'c', 'd', None, None, 8.0, None, None),
 (48, 'b16', 'classify', 'c', 'd', None, None, 8.0, None, None),
 (50, 'b165', 'classify', 'c', 'd', None, None, 8.0, None, None),
 (51, 'b1654', 'classify', 'c', 'd', None, None, 8.0, None, None),
 (54, 'n1', 'predict', 'c', 'd', 4.0, 8.0, None, None, None),
 (55, 'b8', 'classify', 'c', 'd', None, None, 8.0, None, None),
 (57, 'b89', 'classify', 'c', 'd', None, None, 8.0, N

In [15]:
res = doQuery("SELECT * from config_table WHERE model_name = 'c'")

In [16]:
res

[(7, 'c', None, None, None, None, None, None, None, None)]

# Insert Query

In [2]:
insertQuery(('b9898','classify','c','d',8))

In [3]:
res = doQuery('SELECT * from config_table')

In [4]:
res

[(7, 'c', None, None, None, None, None, None, None, None),
 (9, 'fhghf', None, None, None, None, None, None, None, None),
 (11, 'fhf', 'sgs', None, None, None, None, None, None, None),
 (13, 'ff', 'sgs', None, None, None, None, None, None, None),
 (16, 'fdjshsdbjbjhb', None, None, None, None, None, None, None, None),
 (18, 'ashsh', 'b', 'c', "['d','d1']", None, None, 0.9, None, None),
 (19, 'a', 'b', 'c', 'd', None, None, 8.0, None, None),
 (26, 'ard', 'b', 'c', 'd', None, None, 8.0, None, None),
 (30, 'n', 'b', 'c', 'd', 4.0, 8.0, None, None, None),
 (46, 'b167', 'classify', 'c', 'd', None, None, 8.0, None, None),
 (48, 'b16', 'classify', 'c', 'd', None, None, 8.0, None, None),
 (50, 'b165', 'classify', 'c', 'd', None, None, 8.0, None, None),
 (51, 'b1654', 'classify', 'c', 'd', None, None, 8.0, None, None),
 (54, 'n1', 'predict', 'c', 'd', 4.0, 8.0, None, None, None),
 (55, 'b8', 'classify', 'c', 'd', None, None, 8.0, None, None),
 (57, 'b89', 'classify', 'c', 'd', None, None, 8.0, N

In [5]:
insertQuery(('n15','predict','c','d',4,8))

In [6]:
res = doQuery('SELECT * from config_table')

In [5]:
res

[(7, 'c', None, None, None, None, None, None, None, None),
 (9, 'fhghf', None, None, None, None, None, None, None, None),
 (11, 'fhf', 'sgs', None, None, None, None, None, None, None),
 (13, 'ff', 'sgs', None, None, None, None, None, None, None),
 (16, 'fdjshsdbjbjhb', None, None, None, None, None, None, None, None),
 (18, 'ashsh', 'b', 'c', "['d','d1']", None, None, 0.9, None, None),
 (19, 'a', 'b', 'c', 'd', None, None, 8.0, None, None),
 (26, 'ard', 'b', 'c', 'd', None, None, 8.0, None, None),
 (29, 'b', 'b', 'c', 'd', None, None, 8.0, None, None),
 (30, 'n', 'b', 'c', 'd', 4.0, 8.0, None, None, None),
 (46, 'b167', 'classify', 'c', 'd', None, None, 8.0, None, None),
 (48, 'b16', 'classify', 'c', 'd', None, None, 8.0, None, None),
 (50, 'b165', 'classify', 'c', 'd', None, None, 8.0, None, None),
 (51, 'b1654', 'classify', 'c', 'd', None, None, 8.0, None, None),
 (54, 'n1', 'predict', 'c', 'd', 4.0, 8.0, None, None, None)]

# Delete Query

In [7]:
deleteQuery('b')

In [8]:
res = doQuery('SELECT * from config_table')

In [9]:
res

[(7, 'c', None, None, None, None, None, None, None, None),
 (9, 'fhghf', None, None, None, None, None, None, None, None),
 (11, 'fhf', 'sgs', None, None, None, None, None, None, None),
 (13, 'ff', 'sgs', None, None, None, None, None, None, None),
 (16, 'fdjshsdbjbjhb', None, None, None, None, None, None, None, None),
 (18, 'ashsh', 'b', 'c', "['d','d1']", None, None, 0.9, None, None),
 (19, 'a', 'b', 'c', 'd', None, None, 8.0, None, None),
 (26, 'ard', 'b', 'c', 'd', None, None, 8.0, None, None),
 (30, 'n', 'b', 'c', 'd', 4.0, 8.0, None, None, None),
 (46, 'b167', 'classify', 'c', 'd', None, None, 8.0, None, None),
 (48, 'b16', 'classify', 'c', 'd', None, None, 8.0, None, None),
 (50, 'b165', 'classify', 'c', 'd', None, None, 8.0, None, None),
 (51, 'b1654', 'classify', 'c', 'd', None, None, 8.0, None, None),
 (54, 'n1', 'predict', 'c', 'd', 4.0, 8.0, None, None, None),
 (55, 'b8', 'classify', 'c', 'd', None, None, 8.0, None, None),
 (57, 'b89', 'classify', 'c', 'd', None, None, 8.0, N

# Get RowCount

In [10]:
getRowCount('b')

0

In [11]:
getRowCount('c')

1